In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Set up the Selenium WebDriver (assuming Chrome)
driver = webdriver.Chrome()

# Open the target webpage
driver.get('https://sfgov.legistar.com/Legislation.aspx')  # Replace with the actual URL

# Wait for the dropdown menus to be available
wait = WebDriverWait(driver, 10)

#######################
### ADVANCED SEARCH ###
advanced_search_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//input[@value='Advanced search >>>']"))
)
advanced_search_button.click()
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder1_lstMax_Arrow"))  # Adjust this selector
)

# Select the parameters from the dropdown menus
# Replace 'dropdown_id' with the actual ID of the dropdown elements

###################
### ALL RESULTS ###
results_dropdown_arrow = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "ctl00_ContentPlaceHolder1_lstMax_Arrow"))
)
results_dropdown_arrow.click()
all_option = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//li[text()='All']"))
)
all_option.click()

################
### DATE ###
# Wait for the "Date" dropdown to be clickable and click it
date_dropdown_arrow = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "ctl00_ContentPlaceHolder1_lstYearsAdvanced_Arrow"))
)
date_dropdown_arrow.click()

# Wait for the dropdown options to be visible and select the year "2023"
year_option = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//li[text()='All Years']"))
)
year_option.click()

############
### DATE ###
year = 2023
# XPath to select the "between" radio button in for date introduced
radio_button_first_set = driver.find_element(By.XPATH, '//span[@id="ctl00_ContentPlaceHolder1_radFileCreated"]//input[@value="between"]')
radio_button_first_set.click()

# start date for year
# Locate the text box element using an appropriate selector
date_input = driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_txtFileCreated1_dateInput')
# Clear the text box (optional)
date_input.clear()
# Enter the date in mm/dd/yyyy format
date_input.send_keys(f'01/01/{year}')

# end date for year
# Locate the text box element using an appropriate selector
date_input = driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_txtFileCreated2_dateInput')
# Clear the text box (optional)
date_input.clear()
# Enter the date in mm/dd/yyyy format
date_input.send_keys(f'12/31/{year}')

########################
### TYPE = Ordinance ###
type_dropdown_arrow = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "ctl00_ContentPlaceHolder1_lstType_Input"))
)
type_dropdown_arrow.click()

# Wait for the dropdown options to be visible and select the type "Ordinance"
type_option = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//li[text()='Ordinance']"))
)
type_option.click()

##########################
### SEARCH with params ###
# Trigger the search (if there is a search button)
search_button = wait.until(EC.element_to_be_clickable((By.ID, "visibleSearchButton")))
search_button.click()

##################################
### EXTRACT URLS FOR ALL FILES ###
# Wait for the results to load (adjust the selector to the actual results container)
results = wait.until(EC.presence_of_all_elements_located((By.ID, 'ctl00_ContentPlaceHolder1_menuMain')))

# Find all rows in the table
rows = driver.find_elements(By.XPATH, '//tr[@class="rgRow" or @class="rgAltRow"]')

# Function to extract URLs from the current page
def extract_urls():
    rows = driver.find_elements(By.XPATH, '//tr[@class="rgRow" or @class="rgAltRow"]')
    urls = []
    for row in rows:
        link = row.find_element(By.TAG_NAME, 'a')
        if link:
            urls.append(link.get_attribute('href'))
    return urls

# Initialize list to hold all URLs
all_urls = []

# Extract URLs from the first page
all_urls.extend(extract_urls())

# Find the pager links
# same thing is on bottom of page so only find first instance of these classes
pager_links = driver.find_elements(By.XPATH, '(//td[@class="rgPagerCell NumericPages"])[1]//a')


# Loop through each page link and extract URLs
for i in range(1, len(pager_links)):
    pager_links[i].click()
    time.sleep(2)  # Wait for the page to load
    all_urls.extend(extract_urls())
    pager_links = driver.find_elements(By.XPATH, '(//td[@class="rgPagerCell NumericPages"])[1]//a')  # Re-find pager links

# Print all URLs
for url in all_urls:
    print(url)

# Close the WebDriver
driver.quit()


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By

def extract_details(driver):
    details = {}
    
    # Extract file number
    details['file_number'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblFile2").text

    # Extract type
    details['type'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblType2").text

    # Extract introduced date
    details['introduced'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblIntroduced2").text

    # Extract on agenda date
    details['on_agenda'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblOnAgenda2").text

    # Extract enactment date
    details['enactment_date'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblEnactmentDate2").text

    # Extract name
    details['name'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblName2").text

    # Extract status
    details['status'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblStatus2").text

    # Extract in control
    details['in_control'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_hypInControlOf2").text

    # Extract final action date
    details['final_action'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblPassed2").text

    # Extract enactment number
    details['enactment_number'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblEnactmentNumber2").text

    # Extract title
    details['title'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblTitle2").text

    # Extract sponsors
    sponsors = driver.find_elements(By.CSS_SELECTOR, "#ctl00_ContentPlaceHolder1_lblSponsors2 a")
    details['sponsors'] = [sponsor.text for sponsor in sponsors]

    return details

# Initialize WebDriver
driver = webdriver.Chrome()
driver.get("https://sfgov.legistar.com/LegislationDetail.aspx?ID=6458133&GUID=9D8F3348-576B-404B-A2DA-A5AA6E7740C9&Options=Advanced&Search=")

# Wait for the page to load completely
driver.implicitly_wait(10)

# Extract the details
details = extract_details(driver)

# Print the extracted details
for key, value in details.items():
    print(f"{key}: {value}")

# Close the WebDriver
driver.quit()


file_number: 231289
type: Resolution
introduced: 12/29/2023
on_agenda: 
enactment_date: 2/16/2024
name: Liquor License Transfer - 800 Bay Street - Pink House by the Bay
status: Passed
in_control: Clerk of the Board
final_action: 2/16/2024
enactment_number: 056-24
title: Resolution determining that the person-to-person, premise-to-premise transfer of a Type-20 off-sale beer and wine liquor license to Pink House by the Bay LLC, to do business as Pink House by the Bay, located at 800 Bay Street (District 2), will serve the public convenience or necessity of the City and County of San Francisco, in accordance with California Business and Professions Code, Section 23958.4; and requesting that the California Department of Alcoholic Beverage Control impose conditions on the issuance of the license
sponsors: []


In [18]:
import sqlite3
from selenium import webdriver
from selenium.webdriver.common.by import By

def extract_details(driver):
    details = {}
    
    details['file_number'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblFile2").text
    details['type'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblType2").text
    details['introduced'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblIntroduced2").text
    details['on_agenda'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblOnAgenda2").text
    details['enactment_date'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblEnactmentDate2").text
    details['name'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblName2").text
    details['status'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblStatus2").text
    details['in_control'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_hypInControlOf2").text
    details['final_action'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblPassed2").text
    details['enactment_number'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblEnactmentNumber2").text
    details['title'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblTitle2").text
    sponsors = driver.find_elements(By.CSS_SELECTOR, "#ctl00_ContentPlaceHolder1_lblSponsors2 a")
    details['sponsors'] = ', '.join([sponsor.text for sponsor in sponsors])
    
    return details

def store_in_database(data, db_name='legislation.db'):
    # Connect to SQLite database (or create it if it doesn't exist)
    conn = sqlite3.connect(db_name)
    c = conn.cursor()

    # Create table
    c.execute('''CREATE TABLE IF NOT EXISTS legislation
                 (file_number TEXT, type TEXT, introduced TEXT, on_agenda TEXT, enactment_date TEXT,
                  name TEXT, status TEXT, in_control TEXT, final_action TEXT, enactment_number TEXT,
                  title TEXT, sponsors TEXT, url TEXT)''')

    # Insert a row of data
    c.execute('''INSERT INTO legislation (file_number, type, introduced, on_agenda, enactment_date, 
                                          name, status, in_control, final_action, enactment_number, 
                                          title, sponsors, url) 
                 VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', 
                 (data['file_number'], data['type'], data['introduced'], data['on_agenda'], data['enactment_date'],
                  data['name'], data['status'], data['in_control'], data['final_action'], data['enactment_number'],
                  data['title'], data['sponsors'], data['url']))

    # Save (commit) the changes
    conn.commit()

    # Close the connection
    conn.close()
#leg_url = "https://sfgov.legistar.com/LegislationDetail.aspx?ID=6452947&GUID=F7B2AAAE-C341-40CC-B307-E4EB926C02C8&Options=Advanced&Search="
leg_url = "https://sfgov.legistar.com/LegislationDetail.aspx?ID=6452946&GUID=1D3F0E5B-B78B-49F8-BD05-F313A71CE055&Options=Advanced&Search="
#leg_url = "https://sfgov.legistar.com/LegislationDetail.aspx?ID=6452945&GUID=AAE4066E-C232-4B3C-B304-34BF3DDEC843&Options=Advanced&Search="

# Initialize WebDriver
driver = webdriver.Chrome()
driver.get(leg_url)

# Wait for the page to load completely
driver.implicitly_wait(10)

# Extract the details
details = extract_details(driver)
details['url'] = leg_url

# Store the details in the database
store_in_database(details)

# Close the WebDriver
driver.quit()

In [9]:
import pandas as pd

# Load data from SQLite database
conn = sqlite3.connect('legislation.db')
df = pd.read_sql_query("SELECT * FROM legislation", conn)

# Close the connection
conn.close()

# Display the data
# Print each legislation's details in a readable format
for index, row in df.iterrows():
    print("Legislation Details")
    print("-" * 20)
    print(f"File Number:       {row['file_number']}")
    print(f"Type:              {row['type']}")
    print(f"Introduced:        {row['introduced']}")
    print(f"On Agenda:         {row['on_agenda']}")
    print(f"Enactment Date:    {row['enactment_date']}")
    print(f"Name:              {row['name']}")
    print(f"Status:            {row['status']}")
    print(f"In Control:        {row['in_control']}")
    print(f"Final Action:      {row['final_action']}")
    print(f"Enactment Number:  {row['enactment_number']}")
    print(f"Title:             {row['title']}")
    print(f"Sponsors:          {row['sponsors']}")
    print(f"URL:               {row['url']}")
    print("\n" + "=" * 40 + "\n")

Legislation Details
--------------------
File Number:       231273
Type:              Ordinance
Introduced:        12/12/2023
On Agenda:         
Enactment Date:    2/9/2024
Name:              Settlement of Claims, Litigation - SFPUC Water Pipeline Break - Fillmore Street and Green Street - Not to Exceed $7,000,000
Status:            Passed
In Control:        Clerk of the Board
Final Action:      2/9/2024
Enactment Number:  020-24
Title:             Ordinance authorizing the San Francisco Public Utilities Commission (SFPUC) and the Office of the City Attorney to approve settlements of claims arising out of the September 10, 2023, break of the SFPUC's water transmission pipeline at the intersection of Fillmore and Green Streets for amounts exceeding $25,000 per claim, notwithstanding Administrative Code, Section 10.22, et. seq., but not to exceed $1,000,000 per claim, and delegating authority under Charter, Section 6.102(5) for the settlement of related litigation, if any, up to an aggr

In [ ]:
import pandas as pd

# Load data from SQLite database
conn = sqlite3.connect('legislation.db')
df = pd.read_sql_query("SELECT * FROM legislation", conn)

# Close the connection
conn.close()

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By

leg_url = "https://sfgov.legistar.com/LegislationDetail.aspx?ID=6452946&GUID=1D3F0E5B-B78B-49F8-BD05-F313A71CE055&Options=Advanced&Search="

# Initialize WebDriver
driver = webdriver.Chrome()
driver.get(leg_url)

# Wait for the page to load completely
driver.implicitly_wait(10)

def extract_history():
    # Find the table rows
    rows = driver.find_elements(By.XPATH, "//table[@class='rgMasterTable']//tbody//tr")

    # Loop through each row and extract the desired fields
    for row in rows:
        date = row.find_element(By.XPATH, "./td[1]").text
        ver = row.find_element(By.XPATH, "./td[2]").text
        action_by = row.find_element(By.XPATH, "./td[3]").text
        action = row.find_element(By.XPATH, "./td[4]").text
        result = row.find_element(By.XPATH, "./td[5]").text
        
        # Print the extracted details in a readable format
        print("Legislation History Details")
        print("-" * 30)
        print(f"Date:       {date}")
        print(f"Version:    {ver}")
        print(f"Action By:  {action_by}")
        print(f"Action:     {action}")
        print(f"Result:     {result}")
        print("\n" + "=" * 40 + "\n")

# Call the function to extract and print history details
extract_history()

# Don't forget to close the WebDriver
driver.quit()

Legislation History Details
------------------------------
Date:       2/9/2024
Version:    1
Action By:  Mayor
Action:     APPROVED
Result:      


Legislation History Details
------------------------------
Date:       1/30/2024
Version:    1
Action By:  Board of Supervisors
Action:     FINALLY PASSED
Result:     Pass


Legislation History Details
------------------------------
Date:       1/23/2024
Version:    1
Action By:  Board of Supervisors
Action:     PASSED ON FIRST READING
Result:     Pass


Legislation History Details
------------------------------
Date:       1/18/2024
Version:    1
Action By:  Government Audit and Oversight Committee
Action:     RECOMMENDED AS COMMITTEE REPORT
Result:     Pass


Legislation History Details
------------------------------
Date:       12/15/2023
Version:    1
Action By:  Clerk of the Board
Action:     REFERRED TO DEPARTMENT
Result:      


Legislation History Details
------------------------------
Date:       12/12/2023
Version:    1
Action By

In [8]:
import sqlite3
from selenium import webdriver
from selenium.webdriver.common.by import By

def extract_details(driver):
    details = {}
    
    details['file_number'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblFile2").text
    details['type'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblType2").text
    details['introduced'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblIntroduced2").text
    details['on_agenda'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblOnAgenda2").text
    details['enactment_date'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblEnactmentDate2").text
    details['name'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblName2").text
    details['status'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblStatus2").text
    details['in_control'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_hypInControlOf2").text
    details['final_action'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblPassed2").text
    details['enactment_number'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblEnactmentNumber2").text
    details['title'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblTitle2").text
    sponsors = driver.find_elements(By.CSS_SELECTOR, "#ctl00_ContentPlaceHolder1_lblSponsors2 a")
    details['sponsors'] = ', '.join([sponsor.text for sponsor in sponsors])
    
    # Extract history details
    details['history'] = extract_history(driver)
    
    return details

def extract_history(driver):
    history = []
    rows = driver.find_elements(By.XPATH, "//table[@class='rgMasterTable']//tbody//tr")

    for row in rows:
        history_record = {}
        history_record['date'] = row.find_element(By.XPATH, "./td[1]").text
        history_record['ver'] = row.find_element(By.XPATH, "./td[2]").text
        history_record['action_by'] = row.find_element(By.XPATH, "./td[3]").text
        history_record['action'] = row.find_element(By.XPATH, "./td[4]").text
        history_record['result'] = row.find_element(By.XPATH, "./td[5]").text
        history.append(history_record)

    return history

def store_in_database(data, db_name='legislation.db'):
    conn = sqlite3.connect(db_name)
    c = conn.cursor()

    # Create legislation table
    c.execute('''CREATE TABLE IF NOT EXISTS legislation
                 (file_number TEXT, type TEXT, introduced TEXT, on_agenda TEXT, enactment_date TEXT,
                  name TEXT, status TEXT, in_control TEXT, final_action TEXT, enactment_number TEXT,
                  title TEXT, sponsors TEXT, url TEXT)''')

    # Create history table
    c.execute('''CREATE TABLE IF NOT EXISTS history
                 (file_number TEXT, date TEXT, ver TEXT, action_by TEXT, action TEXT, result TEXT)''')

    # Insert data into legislation table
    c.execute('''INSERT INTO legislation (file_number, type, introduced, on_agenda, enactment_date, 
                                          name, status, in_control, final_action, enactment_number, 
                                          title, sponsors, url) 
                 VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', 
                 (data['file_number'], data['type'], data['introduced'], data['on_agenda'], data['enactment_date'],
                  data['name'], data['status'], data['in_control'], data['final_action'], data['enactment_number'],
                  data['title'], data['sponsors'], data['url']))

    # Insert history data into history table
    for record in data['history']:
        c.execute('''INSERT INTO history (file_number, date, ver, action_by, action, result) 
                     VALUES (?, ?, ?, ?, ?, ?)''', 
                     (data['file_number'], record['date'], record['ver'], record['action_by'], record['action'], record['result']))

    conn.commit()
    conn.close()

leg_url = "https://sfgov.legistar.com/LegislationDetail.aspx?ID=6452946&GUID=1D3F0E5B-B78B-49F8-BD05-F313A71CE055&Options=Advanced&Search="

driver = webdriver.Chrome()
driver.get(leg_url)

driver.implicitly_wait(10)

details = extract_details(driver)
details['url'] = leg_url

store_in_database(details)

driver.quit()

In [14]:
import sqlite3
from selenium import webdriver
from selenium.webdriver.common.by import By
from tabulate import tabulate

def extract_details(driver):
    details = {}
    
    details['file_number'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblFile2").text
    details['type'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblType2").text
    details['introduced'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblIntroduced2").text
    details['on_agenda'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblOnAgenda2").text
    details['enactment_date'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblEnactmentDate2").text
    details['name'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblName2").text
    details['status'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblStatus2").text
    details['in_control'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_hypInControlOf2").text
    details['final_action'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblPassed2").text
    details['enactment_number'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblEnactmentNumber2").text
    details['title'] = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_lblTitle2").text
    sponsors = driver.find_elements(By.CSS_SELECTOR, "#ctl00_ContentPlaceHolder1_lblSponsors2 a")
    details['sponsors'] = ', '.join([sponsor.text for sponsor in sponsors])
    
    # Extract history details
    details['history'] = extract_history(driver)
    
    return details

def extract_history(driver):
    history = []
    rows = driver.find_elements(By.XPATH, "//table[@class='rgMasterTable']//tbody//tr")

    for row in rows:
        history_record = {}
        history_record['date'] = row.find_element(By.XPATH, "./td[1]").text
        history_record['ver'] = row.find_element(By.XPATH, "./td[2]").text
        history_record['action_by'] = row.find_element(By.XPATH, "./td[3]").text
        history_record['action'] = row.find_element(By.XPATH, "./td[4]").text
        history_record['result'] = row.find_element(By.XPATH, "./td[5]").text
        history.append(history_record)

    return history

def store_in_database(data, db_name='legislation.db'):
    conn = sqlite3.connect(db_name)
    c = conn.cursor()

    # Create legislation table with an auto-incrementing id
    c.execute('''CREATE TABLE IF NOT EXISTS legislation
                 (id INTEGER PRIMARY KEY AUTOINCREMENT, file_number TEXT, type TEXT, introduced TEXT, 
                  on_agenda TEXT, enactment_date TEXT, name TEXT, status TEXT, in_control TEXT, 
                  final_action TEXT, enactment_number TEXT, title TEXT, sponsors TEXT, url TEXT)''')

    # Create history table with a foreign key to legislation
    c.execute('''CREATE TABLE IF NOT EXISTS history
                 (id INTEGER PRIMARY KEY AUTOINCREMENT, legislation_id INTEGER, date TEXT, ver TEXT, 
                  action_by TEXT, action TEXT, result TEXT, 
                  FOREIGN KEY(legislation_id) REFERENCES legislation(id))''')

    # Insert data into legislation table
    c.execute('''INSERT INTO legislation (file_number, type, introduced, on_agenda, enactment_date, 
                                          name, status, in_control, final_action, enactment_number, 
                                          title, sponsors, url) 
                 VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', 
                 (data['file_number'], data['type'], data['introduced'], data['on_agenda'], data['enactment_date'],
                  data['name'], data['status'], data['in_control'], data['final_action'], data['enactment_number'],
                  data['title'], data['sponsors'], data['url']))

    # Get the id of the newly inserted legislation record
    legislation_id = c.lastrowid

    # Insert history data into history table
    for record in data['history']:
        c.execute('''INSERT INTO history (legislation_id, date, ver, action_by, action, result) 
                     VALUES (?, ?, ?, ?, ?, ?)''', 
                     (legislation_id, record['date'], record['ver'], record['action_by'], record['action'], record['result']))

    conn.commit()
    conn.close()

def get_legislation_data(file_number, db_name='legislation.db'):
    conn = sqlite3.connect(db_name)
    c = conn.cursor()

    # Fetch legislation details
    c.execute('''SELECT * FROM legislation WHERE file_number = ?''', (file_number,))
    legislation = c.fetchone()

    # Fetch history details
    c.execute('''SELECT date, ver, action_by, action, result FROM history WHERE legislation_id = 
                 (SELECT id FROM legislation WHERE file_number = ?)''', (file_number,))
    history = c.fetchall()

    conn.close()
    return legislation, history

def print_legislation_data(file_number, db_name='legislation.db'):
    legislation, history = get_legislation_data(file_number, db_name)

    if legislation:
        # Legislation details
        print("Legislation Details:")
        legislation_headers = ["ID", "File Number", "Type", "Introduced", "On Agenda", "Enactment Date", "Name", 
                               "Status", "In Control", "Final Action", "Enactment Number", "Title", "Sponsors", "URL"]
        print(tabulate([legislation], headers=legislation_headers, tablefmt="pretty"))

        # History details
        if history:
            print("\nHistory:")
            history_headers = ["Date", "Ver", "Action By", "Action", "Result"]
            print(tabulate(history, headers=history_headers, tablefmt="pretty"))
        else:
            print("\nNo history records found.")
    else:
        print(f"No legislation found with file number {file_number}.")

#leg_url = "https://sfgov.legistar.com/LegislationDetail.aspx?ID=6452947&GUID=F7B2AAAE-C341-40CC-B307-E4EB926C02C8&Options=Advanced&Search="
leg_url = "https://sfgov.legistar.com/LegislationDetail.aspx?ID=6452946&GUID=1D3F0E5B-B78B-49F8-BD05-F313A71CE055&Options=Advanced&Search="
#leg_url = "https://sfgov.legistar.com/LegislationDetail.aspx?ID=6452945&GUID=AAE4066E-C232-4B3C-B304-34BF3DDEC843&Options=Advanced&Search="

# Initialize WebDriver
driver = webdriver.Chrome()
driver.get(leg_url)

# Wait for the page to load completely
driver.implicitly_wait(10)

# Extract the details
details = extract_details(driver)
details['url'] = leg_url

# Store the details in the database
store_in_database(details)

# Close the WebDriver
driver.quit()

# Example usage
file_number = '231273'  # Replace with the file number you want to query
print_legislation_data(file_number)

Legislation Details:
+----+-------------+-----------+------------+-----------+----------------+-----------------------------------------------------------------------------------------------------------------------------+--------+--------------------+--------------+------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------